# Защита персональных данных клиентов
___
# Protection of personal data of customers

Необходимо защитить персональные данные клиентов страховой компании. Нужно создать определенный метод преобразования, который не позволит ззлоумышленникам восстановить персональную информацию.

Защиту данных требуется произвести таким образом, чтобы после качество машинного обучения не ухудшалось.
___
It is necessary to protect the personal data of the clients of the insurance company. You need to create a specific conversion method that will not allow attackers to recover personal information.

Data protection needs to be done in such a way that the quality of machine learning does not deteriorate after.

## Загрузка данных
___
## Loading data

In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import r2_score

In [2]:
data = pd.read_csv('insurance.csv')

In [3]:
data.head(5)

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1,41.0,49600.0,1,0
1,0,46.0,38000.0,1,1
2,0,29.0,21000.0,0,0
3,0,21.0,41700.0,2,0
4,1,28.0,26100.0,0,0


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Пол                5000 non-null   int64  
 1   Возраст            5000 non-null   float64
 2   Зарплата           5000 non-null   float64
 3   Члены семьи        5000 non-null   int64  
 4   Страховые выплаты  5000 non-null   int64  
dtypes: float64(2), int64(3)
memory usage: 195.4 KB


In [5]:
data.isna().sum()

Пол                  0
Возраст              0
Зарплата             0
Члены семьи          0
Страховые выплаты    0
dtype: int64

In [6]:
data.duplicated().sum()

153

In [7]:
data.shape[0]

5000

In [8]:
print('Процент дубликатов =', data.duplicated().sum()/data.shape[0])

Процент дубликатов = 0.0306


Процент категорически мал, от дубликатов можно избавляться.
___
The percentage is categorically small, you can get rid of duplicates.

In [9]:
data = data.drop_duplicates().reset_index()

In [10]:
data.duplicated().sum()

0

Дубликаты удалены. Данные предобработаны.
___
Duplicates removed. The data has been preprocessed.

## Алгоритм преобразования
___
## Conversion algorithm

**Алгоритм**

Данные -> Матрица данных умножается на обратную матрицу -> Далее проивзодятся расчеты по алгоритму линейной регрессии
___
**Algorithm**

Data -> The data matrix is multiplied by the inverse matrix -> Next, calculations are made using the linear regression algorithm

**Обоснование**

Таким образом получится видоизменить данные еще перед началом расчетов, что впоследствии помешает восстановить данные по обратным расчетам, так как входные данные были уже изменены.
___
**Rationale**

Thus, it will be possible to modify the data even before the start of the calculations, which will subsequently prevent the restoration of data from reverse calculations, since the input data has already been changed.

## Проверка алгоритма
___
## Algorithm check

Создаем квадратную матрицу.
___
We create a square matrix.

In [11]:
X = np.random.rand(5,5)

X

array([[3.33869180e-01, 7.06422343e-01, 8.75526222e-01, 7.02557110e-01,
        3.12530161e-01],
       [6.50552732e-01, 7.00749978e-01, 9.65814703e-01, 1.61901340e-01,
        5.65524909e-01],
       [1.11509969e-01, 3.38183204e-01, 4.26744152e-04, 9.01463697e-01,
        9.01005877e-01],
       [2.32121322e-01, 6.91127958e-01, 4.46825739e-01, 9.01386994e-01,
        1.28193607e-01],
       [1.17679377e-01, 4.64782140e-01, 5.78932924e-01, 8.63413504e-01,
        7.80083512e-01]])

Проверяем обратимая ли она.
___
Check if it is reversible.

In [12]:
np.linalg.inv(X)

array([[ 39.94823067,  -9.36236195,  19.21521133, -22.05538575,
        -27.78681591],
       [-47.14086286,  12.738166  , -20.61337779,  27.31905262,
         28.97104935],
       [  9.30648401,  -2.50918329,   2.4734152 ,  -5.5911842 ,
         -3.84748402],
       [ 22.65507829,  -6.52557997,  10.15605053, -12.02765716,
        -14.09952849],
       [ -9.92122278,   2.9076471 ,  -3.6936006 ,   4.51210479,
          6.6734744 ]])

Выолним умножение признаков на обратимую матрицу. Для начала определим признаки и целевые значения.
___
Let's perform feature multiplication by an invertible matrix. First, let's define features and target values.

In [13]:
features = data.drop('Страховые выплаты', axis=1)
target = data['Страховые выплаты']

In [14]:
features_mult = features@X

Создадим класс Линейной регрессии.
___
Let's create a Linear Regression class.

In [15]:
class LinearRegression:
    def fit(self, train_features, train_target):
        X = np.concatenate((np.ones((train_features.shape[0], 1)), train_features), axis=1)
        y = train_target
        w = np.linalg.inv(X.T.dot(X)).dot(X.T).dot(y)
        self.w = w[1:]
        self.w0 = w[0]

    def predict(self, test_features):
        return test_features.dot(self.w) + self.w0

Сначала произведем расчеты на основании первоначальных признаков.
___
First, we will make calculations based on the initial features.

In [16]:
model = LinearRegression()
model.fit(features, target)
predictions = model.predict(features)
r2_score_before = r2_score(target, predictions)

Далее произведем те же расчеты на измененных признаках.
___
Next, we will perform the same calculations on the modified features.

In [17]:
model = LinearRegression()
model.fit(features_mult, target)
predictions_mult = model.predict(features_mult)
r2_score_mult = r2_score(target, predictions)

In [18]:
table = [
    [
        r2_score_before,
        r2_score_mult
    ]
]

columns = ["до изменений", "после изменений"]
index = ['r2 score']
table_1 = pd.DataFrame(table, index, columns)
display(table_1)

,до изменений,после изменений
r2 score,0.430218,0.430218


Очевидно отстутствие различий в значениях r2. Качество линейной регрессии не изменилось после преобразования признаков с помощью обратимой матрицы.
___
Obviously, there is no difference in the values of r2. The quality of the linear regression did not change after the feature transformation using the reversible matrix.